In [ ]:
import asyncio
import time
import requests
from indy import anoncreds, did, ledger, pool, wallet, IndyError

import json

async def run():
    print("Send credential offer to device -> started")

    # Set protocol version 2 to work with Indy Node 1.4
    await pool.set_protocol_version(2)

    pool_ = {
        'name': 'pool1',
        'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
    }
    print("Open Pool Ledger: {}".format(pool_['name']))

    try:
        await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
    except IndyError as ex:
        if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
            pass
    pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)


    client = MongoClient(host='supplier_db', port=27017, username='root', password='pass', authSource="admin")
    db = client["device1_db"]

    device1 = db.supplier_tb.find_one({"name": "Device1"})
    
    offer = db.device_tb.find_one({"name": "credential_offer"})
    
    device1['supplier_certificate_cred_offer'] = offer

    supplier_certificate_cred_offer_object = json.loads(device1['supplier_certificate_cred_offer'])

    device1['supplier_certificate_schema_id'] = supplier_certificate_cred_offer_object['schema_id']
    device1['supplier_certificate_cred_def_id'] = supplier_certificate_cred_offer_object['cred_def_id']

    print("\"Device\" -> Get \"Supplier Transcript\" Credential Definition from Ledger")
    (device1['supplier_cred_def_id'], device1['supplier_cred_def']) = \
        await get_cred_def(device1['pool'], device1['did'], device1['supplier_certificate_cred_def_id'])

    print("\"Device\" -> Create \"Transcript\" Credential Request for Supplier")
    (device1['supplier_certificate_cred_request'], device1['supplier_certificate_cred_request_metadata']) = \
        await anoncreds.prover_create_credential_req(device1['wallet'], device1['did'],
                                                     device1['supplier_certificate_cred_offer'], device1['supplier_cred_def'],
                                                     device1['master_secret_id'])


async def get_cred_def(pool_handle, _did, cred_def_id):
    get_cred_def_request = await ledger.build_get_cred_def_request(_did, cred_def_id)
    get_cred_def_response = await ledger.submit_request(pool_handle, get_cred_def_request)
    return await ledger.parse_get_cred_def_response(get_cred_def_response)

if __name__ == '__main__':
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    await run()
    time.sleep(1)  # FIXME waiting for libindy thread complete
